# Enunciado


2. Usando o protocolo  OT construído na questão anterior
    1. Implemente o protocolo $\,\mathsf{sVOLE}\;$ (“subset vectorial oblivious linear evaluation”)
    2. Usando $\,\mathsf{sVOLE}\,$   implemente um protótipo de um protocolo  ZK-sVOLE usando equações polinomiais do 2º grau aleatoriamente geradas.

----------

# PARTE A

## Definição de variáveis

Para definir o protocolo vamos começar por fixar os parâmetros, $\,p\,$, $\,k\,$ e $\,\ell\,$, que determinam a estrutura algébrica onde o protocolo se desenvolve.

In [1]:
p = 3 
k = 5  
l = 8 
lambda_security = 128 
N = 2 * k 

## Estruturas algébricas

In [2]:
F = GF(p)
R.<x> = PolynomialRing(F)
f = R.irreducible_element(k)
E.<a> = GF(p^k, name='a', modulus=f)  

def verifica_base(Z, E):
    F = GF(E.characteristic())
    V = VectorSpace(F, E.degree())  # aqui garantidamente é um VectorSpace
    for j in range(len(Z)):
        subZ = Z[:j] + Z[j+1:]
        mat = matrix(F, [V(E(z)) for z in subZ])
        if mat.rank() < E.degree():
            return False
    return True
    
def gerarZ(E, N):
    F = GF(E.characteristic())
    k = E.degree()
    V = VectorSpace(F, k)  # <-- aqui está garantido que é mesmo um VectorSpace
    tentativas = 0
    while True:
        tentativas += 1
        Z = set()
        while len(Z) < N:
            v = V.random_element()
            if not v.is_zero():
                Z.add(E(v))  # converte vetor para elemento de E
        Z = list(Z)
        if verifica_base(Z, E):
            print(f"Conjunto Z encontrado após {tentativas} tentativas.")
            return Z

Z = gerarZ(E,N)

Conjunto Z encontrado após 2 tentativas.


Esta estrutura algébrica fornece a base para o protocolo OLE, que permite a avaliação linear de forma “oblivious”, sendo uma primitiva fundamental em construção de protocolos criptográficos como OT.

## Oblivious transfer da questão anterior:

In [3]:
import random
import hashlib

def bernoulli(epsilon, n=53):
    """
    Gera uma amostra de Bernoulli B(epsilon) usando a construção de Lebesgue.
    - epsilon: parâmetro da distribuição de Bernoulli (0 < epsilon < 1)
    - n: número de bits para a precisão (default: 53, precisão de um double)
    """
    # Gera a string de bits aleatórios {0,1}^n
    w = [random.randint(0, 1) for _ in range(1, n+1)]
    
    # Calcula o racional de Lebesgue
    w_hat = sum(w[i-1] * 2^(-i) for i in range(1,n+1))
    
    return 1 if w_hat <= epsilon else 0

def bernoulli_lambda(epsilon, n=53, lambda_ = lambda_security):
    return [bernoulli(epsilon,n) for _ in range(lambda_)]

def vector_to_bytes(vector, p):
    """Converte um vetor F_p^ℓ em bytes."""
    byte_array = bytearray()
    for elemento in vector:
        # Assume que elemento está em {0, ..., p-1}
        byte_array.append(elemento % p)  # 1 byte por elemento (se p ≤ 256)
    return bytes(byte_array)

def converter_bytes_para_mensagens(lista_bytes, p=3, l=8, N=10):
    from sage.modules.vector_modn_dense import Vector_modn_dense
    
    mensagens = {}
    F = GF(p)
    
    for n in range(1, N+1):
        if n-1 < len(lista_bytes) and lista_bytes[n-1] is not None:
            # Converte bytes para vetor no F_p^l
            elementos = [F(int(b)) for b in lista_bytes[n-1][:l]]
            mensagens[n] = vector(F, elementos)
        else:
            mensagens[n] = None
    
    return mensagens

def xof(seed: bytes, nbytes: int):
    shake = hashlib.shake_128()
    shake.update(seed)
    return shake.digest(nbytes)  # retorna nbytes de saída
    
def xof_bits(seed: bytes, nbits: int):
    nbytes = (nbits + 7) // 8
    output = xof(seed, nbytes)
    bits = []
    for byte in output:
        for i in range(8):
            bits.append((byte >> (7 - i)) & 1)
            if len(bits) == nbits:
                return bits

In [4]:
class Sender:
    def __init__(self, alpha, l, xof_name="shake128"):
        self.alpha = alpha
        self.l = l
        self.xof_name = xof_name
        self.seed = f"{alpha}:{l}".encode()  # usa α e ℓ como seed
        self.criterion = None
        self.pks = []
        self.N = N  
        self.mensagens = None
        self.r = []
        self.criptogramas = []
        self.t = 100

    def get_vector_ai_ui(self, i, lambda_):
        """Retorna (a_i, u_i) ∈ F₂^λ × F₂, para índice i"""
        # gera (λ + 1) bits para cada i
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]  # bit extra
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        self.criterion = [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def receive_and_verifypks(self, pks):
        self.pks = pks
        for i in range(self.l):
            soma = sum(pks[i][klinha] for klinha in range(self.N)) %2
            _, u_i = self.criterion[i]  # a_i não é necessário aqui
            if soma != u_i:
                print(f"FALHA na linha i={i}: soma={soma}, u_i={u_i}")
                return False
        print("Verificação OK: todas as somas coincidem com u_i")
        return True

    def gerar_r(self, delta, p=0.1):
        while True:
            r = [bernoulli(p) for _ in range(self.l)]
            if sum(r) <= delta:
                self.r = r
                return

    def calcular_criptogramas(self):
        for _ in range(self.t):
            self.gerar_r(128)
            a = [0] * lambda_security  # Agora em F_p
            for i in range(self.l):
                if self.r[i] == 1:
                    a_i, _ = self.criterion[i]
                    a = [(a[j] + a_i[j]) % p for j in range(lambda_security)]
            criptogramas_k = []
            for k in range(1,N+1):
                msg_vector = self.mensagens[k]  # msg_vector é um vetor em F_p^ℓ
                c_k = []
                for elemento in msg_vector:
                    soma = sum(self.r[i] * self.pks[i][k-1] for i in range(self.l)) % 2
                    c_k.append((elemento + soma) % p)
                criptogramas_k.append(c_k)
            self.criptogramas.append((a, criptogramas_k))
            
    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.xof_name: ",self.xof_name)
        print("self.seed: ",self.seed)
        print("self.criterion: ",self.criterion)

In [5]:
class Receiver:
    def __init__(self, eps, b, lambda_, n_mensagens):
        self.alpha = None
        self.l = None
        self.eps = eps
        self.secrets = []
        self.criterion = None
        self.N = n_mensagens
        self.t = None
        self.b = b - 1

    def receive_alpha_l(self, alpha, l):
        self.alpha = alpha
        self.l = l
        self.seed = f"{alpha}:{l}".encode()
        self.criterion = self.get_criterion_sequence(lambda_security)

    def get_vector_ai_ui(self, i, lambda_):
        seed_i = self.seed + f":{i}".encode()
        bits = xof_bits(seed_i, lambda_ + 1)
        a_i = bits[:lambda_]
        u_i = bits[lambda_]
        return a_i, u_i

    def get_criterion_sequence(self, lambda_):
        return [self.get_vector_ai_ui(i, lambda_) for i in range(1, self.l + 1)]

    def generate_N_secrets(self):
        self.secrets = []
        for k in range(self.N):
            if k == self.b:
                self.secrets.append(None)  # s_b = ⊥
            else:
                s_k = bernoulli_lambda(self.eps, lambda_security)
                self.secrets.append(s_k)

    def generate_pks(self):
        # Inicializar t[i][k]
        self.t = [[0 for _ in range(self.N)] for _ in range(self.l)]  
        for i in range(self.l):
            a_i, u_i = self.criterion[i]
            soma = 0
            for k in range(self.N):
                if k != self.b:
                    s_k = self.secrets[k]  # vetor em F₂^λ
                    dot = sum([a_i[j] * s_k[j] for j in range(lambda_security)]) % 2  # produto escalar mod 2
                    e = bernoulli(self.eps)  # bit de erro
                    t_ik = (dot + e) % 2
                    self.t[i][k] = t_ik
                    soma = (soma + t_ik) % 2
                else:
                    self.t[i][k] = None  # inicializa como None por agora
            
            # Calcula t_{i,b} como complemento para somar a u_i
            self.t[i][self.b] = (u_i - soma) % 2

    def recuperar_mensagens(self, a, c):
        mensagens_recuperadas = []
        for k in range(N):
            if k == self.b:
                mensagens_recuperadas.append(None)
            else:
                s_k = self.secrets[k]  # s_k é um vetor em F_p^λ
                a_dot_s = sum(a[j] * s_k[j] for j in range(lambda_security)) % p
                msg_recuperada = [(c[k][i] + a_dot_s) % p for i in range(len(c[k]))]
                mensagens_recuperadas.append(msg_recuperada)
        return mensagens_recuperadas

    def recuperar_mensagens_maioritarias(self, lista_criptogramas):
        t = len(lista_criptogramas)
        resultados_por_k = [[] for _ in range(self.N)]
        
        for i in range(t):
            a, c = lista_criptogramas[i]
            mks = self.recuperar_mensagens(a, c)
            for k in range(self.N):
                resultados_por_k[k].append(tuple(mks[k]) if mks[k] is not None else None)  # Convertemos para tupla
        
        mensagens_finais = []
        for k in range(self.N):
            if k == self.b:
                mensagens_finais.append(None)
            else:
                contagem = {}
                for msg in resultados_por_k[k]:
                    if msg is not None:
                        # Usamos tupla como chave (hashable)
                        msg_key = msg
                        contagem[msg_key] = contagem.get(msg_key, 0) + 1
                # Obtém a mensagem mais frequente
                mensagem_mais_votada = max(contagem.items(), key=lambda x: x[1])[0] if contagem else None
                mensagens_finais.append(bytes(mensagem_mais_votada) if mensagem_mais_votada else None)  # Converte de volta para bytes
        return mensagens_finais

    def print_info(self):
        print("self.alpha: ",self.alpha)
        print("self.l: ",self.l)
        print("self.lambda: ",lambda_security)
        print("self.eps: ",self.eps)
        print("self.secrets: ",self.secrets)
        print("self.criterion: ",self.criterion)
        
    def print_secrets(self):
        print("[")
        for k in range(N):
            if k == self.b:
                print("None")
            else:
                print(self.secrets[k])
        print("]")

## Protocolo sVOLE

O protocolo OLE  simples tem simultaneamente uma questão de segurança e uma questão de eficiência ambas com a mesma proveniência: o número de elementos $\,p^k\,$ do corpo $E$.

- A questão de segurança resulta de, para existir capacidade de verificar o conhecimento de $\,x\in E\,$ ambos os “tags”  $\,\mathsf{m}\,,\,\mathsf{q}\in E\,$ têm de ser suficientemente grandes. Assim o tamanho dessas “tags”, que no protocolo OLE  é  $\,|p|\times k\,$, exige um valor mínimo razoável ($\sim 128\,$bits, ou superior).
- Ainda, em termos de segurança, o tamanho da chave global $\,\Delta\,$ tem de ser sufientemente grande para proteger o protocolo de ataques. Como $\,\Delta\,$ é um valor gerado de $\,E\,$ o factor “segurança de $\Delta$”  exige também um valor mínimo para $\,p^k\,$.
- A questão de eficiência resulta do facto de o número $\,N\,$ de mensagens usadas no protocolo OT  e usadas nas somas que calculam os “tags” ser também $\,p^k\,$. A eficiência exige que $\,N\,$ não possa ser muito grande.


Para criar uma solução que responda  simultaneamente a todas estas questões, a partir de um factor de segurança $\,\lambda\,$ e vamos seguir os seguintes objectivos

1. Os “tags” $\,\mathsf{m}\,$e $\,\mathsf{q}\,$ passam a ser vetores com $\,\ell\,$ componentes em $\,E\,$; como  elementos de $\,E^\ell\,$ o  o seu tamanho é substancialmente aumentado para  $\,|p|\times k\times \ell\,$;  se a dimensão $\,\ell\,$ for suficientemente grande, mesmo com $|p|$ e $\,k\,$ relativamente pequenos 
2. Os índices $\,z\,$ que identificam os elementos $\,t_z\,$ usados no protocolo OT   formam um subconjunto  $\,\mathcal{Z}\,\equiv\,\{z_1,z_2,\cdots\,z_N\}\,\subset E\,$ com $\,N  = \mathsf{poly}(\lambda)$  elementos.  
    O valor de $\,N\,$ e os elementos $\,\{z_n\}_{n\in[N]}\,$ são escolhidos de tal forma que, para todo $\,n\in [N]\,$ o conjunto $\,\mathcal{Z}\setminus\{z_n\}\,$ é uma base de $\,\mathbb{F}_{p^k}\,$ visto como espaço vectorial sobre $\,\mathbb{F}_p$.  
    Concretamente, cada $\,a\in E\,$ é uma combinação linear, com coeficientes em $\,F\,,$ dos elementos de $\,\mathcal{Z}\,$ e , cada $\,z_n\in\mathcal{Z}\,$ é uma combinação linear dos elementos de $\,\mathcal{Z}\setminus\{z_n\}\,$.
3. Os valores a autenticar $\,\overline{x}\,=\,\langle x_1, x_2,\cdots,x_\ell\rangle\,$ passam a ser também vectores de domensão $\,\ell\,$; isto é $\,\overline{x}\in F^{\ell}$.
4. A chave global $\,\Delta\,$ é escolhida, em cada instância do protocolo, dentro do  conjunto $\,\mathcal{Z}\,$ através da seleção de um índice $\,j\gets [N]\,$, e depois escolhendo $\,\Delta \equiv z_j\,$.
                                              $\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}$
5. Os valores $\,t_z\in F\,\,$ do protocolo OLE são substituídos por vectores; assim, para todo $\,i\in[\ell]\,$,  define-se um vector  $\,\overline{t}_i = \{t_{i,n}\}_{n\in[N]}\in\,F^\ell\,$ em que, 
        1. para $\,n>1\,$ , tem-se $\,t_{i,n}\,\gets\,F\,$ é gerado pseudo-aleatoriamente
        2. para  $\,n=1\,$  tem-se  $t_{i,1}\,\gets\, x_i - \sum_{n=2}^N\,t_{i,n}$.
    Isto assegura que, para todo $\,i\in[\ell]\,$, se verifica $\,x_i \,=\,\sum_{n\in[N]}\,t_{i,n}$



6. Ambos os agentes, Prover e Verifier  conhecem os parâmetros  $\,p,k,\ell\,$ e o parâmetro de segurança $\,\lambda\,$. Conhecem também $\,\mathcal{Z} \subset \mathbb{F}_{p^k}\,$ e a sua cardinalidade $\,N=O(\mathsf{poly}(\lambda))\,$, como especificámos atrás.

### Prover sVOLE

Este agente conhece um vector $\,\overline{x}\in F^\ell\,$ , que é sua informação privada, e procede do seguinte modo 

a. para todo $\,i\in [\ell]$ \
   i. Gera o vector $\,\overline{t}_i\,$  como está indicado no ponto 5. acima, \
   ii. Disponibiliza $\,N\,$ mensagens $\,\{m_{i,n} \gets \sigma(t_{i,n})\}_{n\in [N]}\,$   para transferência num protocolo “oblivious transfer”  $\,{N\choose{N-1}}$ . \
   iii. Calcula o “tag”  $\;\mathsf{m}_i\,\gets\,\sum_{n\in[N]}\,(-z_n)\cdot t_{i,n}\,$.

b. Agrega num vector $\;\overline{\mathsf{m}}\in E^\ell\;$ os vários “tags”  $\,\{\mathsf{m}_i\}_{i\in [\ell]}$

In [6]:
class Prover_sVOLE:
    def __init__(self):
        self.vectorX = vector(F, [F.random_element() for _ in range(l)])
        self.t_matrix = {}
        self.messages = []
        self.sender = Sender(alpha="alpha123", l=10)
        self.tag = None

    #passo a.i
    def gerar_tz(self):
        for i in range(l):  
            t_i = {n: F.random_element() for n in range(2, N + 1)}
            
            t_i[1] = self.vectorX[i] - sum(t_i[n] for n in range(2, N + 1))
            
            # debug
            assert self.vectorX[i] == sum(t_i[n] for n in range(1, N + 1)), \
                   f"Erro na soma para x[{i}]"

            #sort simples
            t_keys = list(t_i.keys())
            t_keys.sort()
            
            self.t_matrix[i] = {i:t_i[i] for i in t_keys}

    def mostrar_estrutura(self):
        print(f"Vetor privado x: {self.vectorX}")
        print("\nMatriz t_{i,n}:")
        for i in range(min(3, l)):  # Mostra apenas as 3 primeiras linhas para exemplo
            print(f"i={i}: {self.t_matrix.get(i, 'Não gerado')}")

    #ponto a.ii
    def gen_messages(self):
        self.messages = {
            n: vector(F, [self.t_matrix[i][n] for i in range(l)])
            for n in range(1, N + 1)
        }
        print("Mensagens geradas:")
        for i in range(1,l+1):
            print(f"{i}: {self.messages[i]}")

    # Função sigma que codigica um elemento de F como string de bits
    def _sigma(self, x):
        bit_length = (p - 1).bit_length()
        
        if x in F: 
            return format(int(x), f'0{bit_length}b')
        elif isinstance(x, list) and all(y in F for y in x):
            return ''.join([format(int(y), f'0{bit_length}b') for y in x])
        else:
            raise ValueError("x deve ser um elemento de F ou uma lista de elementos de F")

    #ponto a.iii / b
    def gerar_tag(self):
        self.tag = []
        for i in range(l):
            soma = E(0)
            for n in range(1, N + 1):
                z_n = Z[n - 1]
                t_in = self.t_matrix[i][n]
                soma += (-z_n) * E(t_in)
            self.tag.append(soma)
        print("Tag calculado:", self.tag)
            
prover_sVOLE = Prover_sVOLE()
prover_sVOLE.gerar_tz()
prover_sVOLE.mostrar_estrutura()
prover_sVOLE.gen_messages()

from sage.modules.vector_modn_dense import Vector_modn_dense

def dicionario_para_vetores(messages_dict, p, l):
    vetores = {}
    for n in messages_dict:
        vetor_n = messages_dict[n]
        elementos = []
        for i in range(l):
            elemento = vetor_n[i]
            # Se for um vetor, pega o primeiro elemento (ajuste conforme necessário)
            if hasattr(elemento, '__len__'):  # Verifica se é um objeto tipo sequência
                elemento = elemento[0]
            elementos.append(F(elemento))
        vetores[n] = vector(F, elementos)
    return vetores

# Uso:
vetores_OT = dicionario_para_vetores(prover_sVOLE.messages, p, l)
print("vetores_OT: ",vetores_OT)
prover_sVOLE.sender.mensagens = vetores_OT
prover_sVOLE.gerar_tag()

Vetor privado x: (0, 2, 2, 1, 1, 1, 2, 2)

Matriz t_{i,n}:
i=0: {1: 2, 2: 0, 3: 2, 4: 1, 5: 0, 6: 2, 7: 2, 8: 2, 9: 1, 10: 0}
i=1: {1: 1, 2: 2, 3: 2, 4: 2, 5: 2, 6: 0, 7: 2, 8: 1, 9: 2, 10: 0}
i=2: {1: 2, 2: 2, 3: 2, 4: 0, 5: 0, 6: 2, 7: 1, 8: 1, 9: 1, 10: 0}
Mensagens geradas:
1: (2, 1, 2, 1, 0, 2, 0, 2)
2: (0, 2, 2, 2, 2, 0, 1, 1)
3: (2, 2, 2, 0, 1, 1, 1, 2)
4: (1, 2, 0, 1, 0, 0, 1, 1)
5: (0, 2, 0, 1, 0, 1, 0, 0)
6: (2, 0, 2, 2, 2, 0, 1, 1)
7: (2, 2, 1, 2, 1, 0, 0, 1)
8: (2, 1, 1, 1, 1, 2, 0, 2)
vetores_OT:  {1: (2, 1, 2, 1, 0, 2, 0, 2), 2: (0, 2, 2, 2, 2, 0, 1, 1), 3: (2, 2, 2, 0, 1, 1, 1, 2), 4: (1, 2, 0, 1, 0, 0, 1, 1), 5: (0, 2, 0, 1, 0, 1, 0, 0), 6: (2, 0, 2, 2, 2, 0, 1, 1), 7: (2, 2, 1, 2, 1, 0, 0, 1), 8: (2, 1, 1, 1, 1, 2, 0, 2), 9: (1, 2, 1, 1, 0, 2, 2, 0), 10: (0, 0, 0, 2, 0, 2, 2, 1)}
Tag calculado: [a^4 + 2*a^2 + 2*a + 2, a^4 + a^3 + 2*a, 2, a^4 + 2*a^2 + 2*a + 1, a^4 + a^3 + a + 1, a^3 + 2*a + 1, a^4 + 2*a^3 + 2*a^2 + 2*a + 1, 2*a^4 + a^3 + a^2 + a + 2]


### Verifier sVOLE

Verifier

- para todo 
    1. Gera $\,\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}\;$ que passa a ser designada por chave global. 
    2. para todo $\,i\in [\ell]$
        1. No $i$-ésimo  protocolo $\,{N}\choose{N-1}\,$-OT  inicializado pelo Prover, transfere as $\,N-1\,$ mensagens $\{m_{i,n}\}_{n\in [N]\setminus\{j\}}$ 
        2. Calcula  $\quad\mathsf{q}_i\,\gets\,\sum_{n\in [N]\setminus{\{j\}}}\,(\Delta-z_n)\cdot\sigma^{-1}(m_{i,n})$
    3. Agrega num único vector $\,\overline{\mathsf{q}}\in E^\ell\,$  todas as “tags” $\,\{\mathsf{q}_i\}_{i\in[\ell]}$

In [7]:
class Verifier_sVOLE:
    def __init__(self):
        self.j = random.randint(1, N)
        self.Delta = Z[self.j-1]
        self.q = []
        self.receiver = Receiver(0.1,self.j,lambda_security,N)

    # Passo 1: Escolhe Delta = z_j, onde z_j ∈ Z
    def gerar_delta(self):
        self.Delta = Z[self.j-1]
        print(f"Δ = {self.Delta} (índice j = {self.j})")

    # Passo 2.1: Obtém mensagens do Prover exceto a do índice j
    def participar_OT(self, mensagens):
        self.mensagens_OT = {
            n: mensagens[n] 
            for n in mensagens 
            if n != self.j and mensagens[n] is not None
        }
        print("Mensagens OT (excluindo j):", self.mensagens_OT)

    # Passo 2.2: Calcula q_i 
    def calcular_q_i(self, i):
        soma = E(0)
        for n in self.mensagens_OT:
            if n == self.j:  # Pula o índice j (Δ)
                continue
            z_n = Z[n - 1]
            m_in = self.mensagens_OT[n][i]  # Acesso correto: mensagem[n][i]
            t_in = self._sigma_inversa(m_in)
            soma += (self.Delta - z_n) * E(t_in)
        return soma

    # Passo 3: Agrega todos q_i em q̄
    def calcular_q(self):
        self.q = [self.calcular_q_i(i) for i in range(l)]
        print("Vetor q̄:", self.q)
        
    # Decodifica mensagem (bits para elemento de F)
    def _sigma_inversa(self, bit_str):
        from sage.modules.vector_modn_dense import Vector_modn_dense
        
        # Se for string binária
        if isinstance(bit_str, str):
            return F(int(bit_str, 2))
        
        # Se for um vetor do Sage ou sequência
        if hasattr(bit_str, '__getitem__'):
            return F(bit_str[0])  # Pega o primeiro elemento
        
        # Se já for um elemento conversível
        return F(bit_str)

    def verificar_relacao(self, prover_tag, prover_x):
        resultados = []
        for i in range(l):
            q_i = self.q[i]
            x_i = E(prover_x[i])
            m_i = prover_tag[i]
            lado_direito = x_i * self.Delta + m_i
            
            print(f"i={i}: q_i = {q_i}")
            print(f"x_i*Δ + m_i = {x_i}*{self.Delta} + {m_i} = {lado_direito}")
            print(f"Resultado: {q_i == lado_direito}")
            
            resultados.append(q_i == lado_direito)
        return all(resultados)

In [8]:
# Inicializa Verifier
verifier_sVOLE = Verifier_sVOLE()

# Gera Δ (Passo 1)
verifier_sVOLE.gerar_delta()

# Participa do OT (Passo 2.1)
prover_sVOLE.sender.get_criterion_sequence(lambda_security)
verifier_sVOLE.receiver.receive_alpha_l("alpha123", 10)
verifier_sVOLE.receiver.generate_N_secrets()
verifier_sVOLE.receiver.generate_pks()
prover_sVOLE.sender.receive_and_verifypks(verifier_sVOLE.receiver.t)
prover_sVOLE.sender.calcular_criptogramas()


mensagens_decodificadas = verifier_sVOLE.receiver.recuperar_mensagens_maioritarias(prover_sVOLE.sender.criptogramas)
mensagens_decodificadas = converter_bytes_para_mensagens(mensagens_decodificadas)


for n in prover_sVOLE.sender.mensagens:
    if verifier_sVOLE.j != n:
        if prover_sVOLE.sender.mensagens[n] == mensagens_decodificadas[n]:
            print(f"Mensagem n={n} bem decifrada")
        else:
            print(f"Diferença em n={n}:")
            print(f"Original: {prover_sVOLE.sender.mensagens[n]}")
            print(f"Decodificado: {mensagens_decodificadas[n]}")
            print(f"Iguais? {prover_sVOLE.sender.mensagens[n] == mensagens_decodificadas[n]}")
    else:
        if mensagens_decodificadas[n] is None:
            print(f"Mensagem n={n} (índice b) foi corretamente omitida")

verifier_sVOLE.participar_OT(mensagens_decodificadas)

# Calcula q̄ (Passos 2.2 e 3)
verifier_sVOLE.calcular_q()

# Verifica relação q̄ = x̄ * Δ + m̄ (Opcional)
resultado = verifier_sVOLE.verificar_relacao(prover_sVOLE.tag, prover_sVOLE.vectorX)
print("Verificação bem-sucedida?", resultado)

Δ = 2*a^4 + 2*a^3 + 2*a^2 + 1 (índice j = 5)
Verificação OK: todas as somas coincidem com u_i
Mensagem n=1 bem decifrada
Mensagem n=2 bem decifrada
Mensagem n=3 bem decifrada
Mensagem n=4 bem decifrada
Mensagem n=5 (índice b) foi corretamente omitida
Mensagem n=6 bem decifrada
Mensagem n=7 bem decifrada
Mensagem n=8 bem decifrada
Mensagem n=9 bem decifrada
Mensagem n=10 bem decifrada
Mensagens OT (excluindo j): {1: (2, 1, 2, 1, 0, 2, 0, 2), 2: (0, 2, 2, 2, 2, 0, 1, 1), 3: (2, 2, 2, 0, 1, 1, 1, 2), 4: (1, 2, 0, 1, 0, 0, 1, 1), 6: (2, 0, 2, 2, 2, 0, 1, 1), 7: (2, 2, 1, 2, 1, 0, 0, 1), 8: (2, 1, 1, 1, 1, 2, 0, 2), 9: (1, 2, 1, 1, 0, 2, 2, 0), 10: (0, 0, 0, 2, 0, 2, 2, 1)}
Vetor q̄: [a^4 + 2*a^2 + 2*a + 2, 2*a^4 + 2*a^3 + a^2 + 2*a + 2, a^4 + a^3 + a^2 + 1, 2*a^3 + a^2 + 2*a + 2, 2*a^2 + a + 2, 2*a^4 + 2*a^2 + 2*a + 2, 2*a^4 + 2*a, 2*a^3 + 2*a^2 + a + 1]
i=0: q_i = a^4 + 2*a^2 + 2*a + 2
x_i*Δ + m_i = 0*2*a^4 + 2*a^3 + 2*a^2 + 1 + a^4 + 2*a^2 + 2*a + 2 = a^4 + 2*a^2 + 2*a + 2
Resultado: Tru